In [ ]:
import os
import json
import random
from glob import glob
from typing import Mapping, Literal, Any

import textworld.gym
from textworld import EnvInfos, Agent

import numpy as np
import pandas as pd
import ollama

from IPython.display import clear_output

from langgraph.graph import MessagesState
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, ToolMessage

from langgraph.graph import START, StateGraph
from langgraph.prebuilt import ToolNode, tools_condition
from IPython.display import Image, display

from langchain_core.tools import tool

from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama

from langgraph.checkpoint.memory import MemorySaver

from langgraph.graph import END

In [ ]:
np.random.seed(42)
random.seed(42)
SEEDS = [random.randint(0, 1000) for _ in range(5)]
print(SEEDS)
BPMNS = sorted([x for x in os.listdir("bpmn") if x.endswith(".bpmn")])
print(BPMNS)
BPMN_PATHS = [os.path.join("bpmn", x) for x in BPMNS]
print(BPMN_PATHS)

if not os.path.isdir("rules"):
    os.makedirs("rules")

In [ ]:
for bpmn_path in BPMN_PATHS:
    for seed in SEEDS:    
        llm = ChatOpenAI(
            model="gpt-4o",
            temperature=0,
            seed=seed
        )
        process_bpmn = ""
        with open(bpmn_path, "r") as f:
            process_bpmn = f.read()
        process_rule = llm.invoke(f"""You are an assistant that converts business process models defined with the business process model notation in XML into process steps an LLM agent will automatically execute to complete the process. 

        Convert the BPMN XML into unambiguous and pragmatic step that include important step information, decision rules, and the additional text annotations.

        - The steps need to be designed as clear, unambiguous instructions for an LLM agent.
        - Convert XOR statements to if-else statements based on the XOR gateway decision rules.
        - Nest every step of an if path within the if statement and every else path step within an else statement.
        - Linearize parallelization gateways into sequential steps.
        - Include all additional text annotations from the BPMN XML in the respective step instruction.
        - Ignore all process parts that are not done in the LLM Agent lane.
        - Hold the formatting of the steps as simple as possible.

        Convert the following XML BPMN modeling according to the instructions above:

        {process_bpmn}

        Do not include receiving the error message as the first step.""").content
        rule_path = os.path.join("rules", f"{bpmn_path.split('/')[-1].replace('.bpmn', '')}")
        if not os.path.isdir(rule_path):
            os.makedirs(rule_path)
        with open(os.path.join(rule_path, f"{seed}.md"), "w") as f:
            f.write(process_rule)